# Find Unique
 - Deliverables:
     - findUnique.py - 50 points ( We will test this code)
     - Notebook with Inspection materials and Results ( 5 pts )
     - required input from STDIN
     - required output to STDOUT
     - output must be sorted by tRNA header

## Sets and tRNA

Mitochondrial tRNA are encoded in the mitochondrial genome and account for ~10% of the coding space, yet over 50% of mitochondrial genomic disease have their origin in this molecule. These molecules are transcribed, processed, modified, and ultimately folded to produce these mature adapters between the mitochondrial transcription and translation processes. Defects in mt.tRNA maturation reflect additional disease states .. if only we could identify those mutations. We are working on such a device, though we first need to use it to identify abundance of these molecules in a mixed population. Are there any unique subsequences among the 22 mt.tRNA that can be used as "tags"? If so, we can count those tags to assess abundance.

## Assignment

Write a python command line program that reads a file of fasta sequences from STDIN, finds the unique subsequences that occur in each single tRNA such that no members of this set occur among any of the other tRNA sets. Each of those 22 sets should be minimized, such that no member of that unique subsequence set is a substring of any other member of this set. [ Unique and Essential]

As an example, let's say that both ACG and AAACGA are in a unique set. Since ACG is a substring of AAACGA we would remove AAACGA. [ ACG is Essential ]

Use Python sets for this assignment[__required__]. Not only will your code be smaller, but it will be more likely to work. The union, intersection and difference operators will be quite useful.

## Rough design plan...

1) compute the set of all substrings from each tRNA sequence. [powerset] I will refer to a set of substrings as a tRNAset.

2) for each tRNAset, compute the union of all other tRNA sets, and then remove that union from the current tRNAset. Notice that this union operation finds all of the substrings from all other tRNA. If any of those are present in your current tRNA, then they are not unique ! [ Unique]

3) for each unique tRNAset, it now contains the truly unique ones, along with any extensions of that subsequence. If, for example, it was found that G only occurred in a single tRNA, then adding an A onto that G must also be unique because it has a G in it. We only want the minimal form.. G. [ Essential]

4) Remove spaces from the header line before sorting and printing. This will make your output a little prettier.

5) make sure to remove any alignment characters from your initial sequences. These characters are periods(.), underscores(\_), or dashes(-) . You will find many new characters in the sequence other than {ACGU} - leave these in place, they are modified bases.

## Report
Print a report that contains items as follows. 

 - Line 1: the tRNA name
 - Line 2: the tRNA sequence ( with the alignment characters removed)
 - lines 3-80 or so, each unique element.

These unique elements need to be ordered by their starting position in the tRNA sequence, and should be spaced over so they are directly under that subsequence in the original tRNA. This looks like an alignment, but you can find where it belongs by using the string.find() method. Include dots in all positions to the left, serving as alignment guides for your reader. [ see sample output below ]

Do this for all of the 22 tRNA sequences.

Print the tRNA out as above, sorted by the header line.  

## Hints:

__use sets and the set operators - this is required !__

your final code will be under 100 lines.

Do most of your coding using methods defined in a class and then instantiated as objects. 

The sequences include characters that are just alignment characters. They are not part of the sequence and must be removed. [-\_\.] are alignment characters. 

When removing items from a tRNAset, don't do this while iterating through that set. Also, when building a unique set you will need the original contents of all other tRNAsets. So.. build a new set to keep the unique contents, or keep track of the elements you intend to delete later. Notice that you build the union of all other tRNA, and this happens 22 times - these unions are all distinct from each other. Example, consider 4 sets, A,B,C,D.  we would compute the set B, C, D to use against set A, and we would compute the set A, C, D to use against B. A and B need to not change while we are doing this computation.

There are cases where a unique element is present multiple times in a single tRNA. This sequence is unique to this tRNA and should be reported in its multiple locations ( same sequence on multiple lines of the output).

This is a command line program, though it does not have any optional parameters. You really don't need the commandline class. Your input comes from STDIN and output goes to STDOUT. Use the FastaReader for input and print statement for output. If you are going to use jupyter to develop your code, you can add a filename to fastareader for your testing.

## Submission

Submit code using Canvas. As always, work with your group and write your own code.

## Inspection Intro
I want the team to focus on readability and accuracey of the output results. I want feedback on if my current code is easy to follow and understand or if not, what can I do to improve the code. I would like input on whether the outputs are correct or if there are still bugs I have yet to catch.

In [1]:
#!/usr/bin/env python3
# Name: Fan Xia (fxia)
# Group Members: Jorge Gomez Ortega (jgomezor), Nishal Naicker (nnaicker), and Rachana Baskar(rbaskar)

import sys
class FastAreader :
    ''' 
    Define objects to read FastA files.
    
    instantiation: 
    thisReader = FastAreader ('testTiny.fa')
    usage:
    for head, seq in thisReader.readFasta():
        print (head,seq)
    '''
    def __init__ (self, fname=''):
        '''contructor: saves attribute fname '''
        self.fname = fname
            
    def doOpen (self):
        ''' Handle file opens, allowing STDIN.'''
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)
        
    def readFasta (self):
        ''' Read an entire FastA record and return the sequence header/sequence'''
        header = ''
        sequence = ''
        
        with self.doOpen() as fileH:
            
            header = ''
            sequence = ''
            
            # skip to first fasta header
            line = fileH.readline()
            while not line.startswith('>') :
                if not line: # we are at EOF
                    return header, sequence
                line = fileH.readline()
            header = line[1:].rstrip()

            for line in fileH:
                if line.startswith ('>'):
                    yield header,sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else :
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header,sequence
        
        
####################### t_RNA ###########################################

'''
Program overview:
    
    
Creates a t_RNA class object that stores the sequence, header, powerset, set of uniques, and set of essentials. 
Prints out the the essential sequences in ascending order of the index where it is found in the sequence.
    
'''

class t_RNA:
    
    ''' building a new t_RNA'''
    
    tRNAList = [] # list of tRNA objects
    
    def __init__(self, inSeq, header):
        
        '''creating a constructor for the class'''
        
        t_RNA.tRNAList.append(self)
        self.inSeq = inSeq
        self.header = header
        
        self.duplicates = set()
        
        self.powerset = set()
        self.uniques = set()
        self.essentials = set()
        
    
    def makePowerset(self):
        '''method that creates a powerset of the inSeq'''
        for a in range(0,len(self.inSeq)):
            for b in range(a+1, len(self.inSeq)+1):
                sequence = self.inSeq[a:b] # creating each sequence to be added to the powerset                    
                if sequence in self.powerset:
                    self.duplicates.add(sequence) # if sequence already exists in the powerset, add it to a duplicates set to be removed later           
                self.powerset.add(sequence) # add sequence to powerset
        
    def findUnique(self):
        
        '''method that finds the unique sequences within the powerset'''
        
        psetList = [] # list of powersets for each object
        
        for obj in t_RNA.tRNAList: # create a list of 22 powersets, one for each object created
            psetList.append(obj.powerset)
        
        psetList.remove(self.powerset) # remove the current powerset we are comparing with
        union = psetList[0].union(*psetList) # finding the union of the other 21 tRNA powersets
        uniquetRNA = (self.powerset).difference(union) # finding the difference of this tRNA and the other 21 tRNAs
        uniquetRNA.difference(self.duplicates) # remove the duplicate sequences from our uniquetRNA set 
        
        for unique in uniquetRNA: # for every sequence in the uniquetRNA set, add each sequence to self.uniques
            self.uniques.add(unique)
            
    def findEssential(self):
        '''method that takes the set of uniques and determines which are essential'''
        
        nonEssentials = set() # set to store non-essential sequences

        for sequence in self.uniques:
            if sequence[:-1] in self.uniques or sequence[1:] in self.uniques: 
                nonEssentials.add(sequence)
            
        essentialtRNA = (self.uniques).difference(nonEssentials)
        
        for essential in essentialtRNA:
            self.essentials.add(essential)
            
    
    def printEssentials(self):
        '''method that prints the essentials by ascending order of the index in the original sequence'''
        
        essIndexList = [] # creating a list of essential indices that will store (index,sequence) for each sequence in self.essentials
        
        for sequence in self.essentials:
            index = self.inSeq.find(sequence)
            essIndexList.append((index,sequence)) 
            
        essIndexList.sort(key = lambda x:x[0]) # sort by ascending order of the index
        
        for indexSeq in essIndexList:
            print('.'*indexSeq[0]+indexSeq[1])
                    


########################################################################
# Main
# Here is the main program
# 
########################################################################

'''

Program overview:

Builds a new t_RNA class object for every tRNA identified in a file and prints out the header, sequence, and then
all of the essential sequences in ascending order by index in the original sequence.

Ex.

Input in command line: python3 findOrfs.py  <'bos-tRNA-7.fa'

Output (for one tRNA):

tRNA|Ala|UGC|Bostaurus|mitochondrial
GAGGAUUU"LCUUAAUUAAAGULGPUGAUUUGCAUPCAAUUGAUGUAAGGUGPAGUCUUGCAAUCCUUACCA
GAGGA
..GGAU
...GAUUU"
......UU"LC
.........LCUUAAU
...........UUAAUUA
..............AUUAAA
...............UUAAAG
...................AGUL
.....................ULG
......................LGP
.......................GPUGA
........................PUGAU
...........................AUUUG
............................UUUGC
..............................UGCAU
.................................AUPC
...................................PCAA
....................................CAAUUG
.....................................AAUUGA
......................................AUUGAUG
.........................................GAUGUA
...........................................UGUAAGG
...............................................AGGUG
...................................................GPA
....................................................PAGU
.....................................................AGUCUU
......................................................GUCUUG
........................................................CUUGC
.........................................................UUGCAA
...........................................................GCAAU
.............................................................AAUC
..............................................................AUCCUUA
................................................................CCUUAC

'''

def main(inCL=None):
    
    ''' prints out header, sequence, and the essential tRNA sequences '''
    
    myReader = FastAreader(inCL)
    
    for head, seq in myReader.readFasta() : # creates a new object with a powerset for every tRNA
        newSeq = seq.replace('.','').replace('-','').replace('_','')
        newHead = head.replace(' ','')
        newtRNA = t_RNA(newSeq, newHead)
        newtRNA.makePowerset()
        
    t_RNA.tRNAList.sort(key = lambda x:x.header) # sort the objects by the header
    
    for tRNA in t_RNA.tRNAList: # for each object in the tRNAList in the t_RNA class, print the header, sequence, and essentials
        tRNA.findUnique()
        tRNA.findEssential()
        print(tRNA.header)
        print(tRNA.inSeq)
        tRNA.printEssentials()

if __name__ == "__main__":
    main('bos-tRNA-7.fa')  


tRNA|Ala|UGC|Bostaurus|mitochondrial
GAGGAUUU"LCUUAAUUAAAGULGPUGAUUUGCAUPCAAUUGAUGUAAGGUGPAGUCUUGCAAUCCUUACCA
GAGGA
..GGAU
...GAUUU"
......UU"LC
.........LCUUAAU
...........UUAAUUA
..............AUUAAA
...............UUAAAG
...................AGUL
.....................ULG
......................LGP
.......................GPUGA
........................PUGAU
...........................AUUUG
............................UUUGC
..............................UGCAU
.................................AUPC
...................................PCAA
....................................CAAUUG
.....................................AAUUGA
......................................AUUGAUG
.........................................GAUGUA
...........................................UGUAAGG
...............................................AGGUG
...................................................GPA
....................................................PAGU
.....................................................AGUCUU
....

...............................................GGUUUA
................................................GUUUAU
....................................................AUAUCC
.....................................................UAUCCU
......................................................AUCCUUC
..........................................................UUCCC
...........................................................UCCCG
..............................................................CGU
................................................................UACUA
.................................................................ACUAC
tRNA|Leu|UAG|Bostaurus|mitochondrial
ACUUUUAA"LGAUAGUAGDUUAUCCLPPGGPCUUAGKAACCAAAAAAUUGGUGCAACUCCAAAUAAAAGUACCA
ACUUU
.CUUUUA
..UUUUAA
......AA"
.......A"LG
.........LGA
..........GAUAGU
................AGD
..................DUU
...................UUAUC
.......................CCL
.........................LP
............................GGP
.............................GPC


tRNA|Ser|UGA|Bostaurus|mitochondrial
GAGAGAGACAUAGAGGDUAUGAPGPPGG'UUGA+ACCAAUAGUAGGGGGUPCG"UUCCUUCCUUUCUUACCA
GAGAGA
.AGAGAG
...AGAGAC
.....AGACA
......GACAU
.......ACAUA
........CAUAG
.............AGGD
..............GGDU
................DUA
....................GAP
......................PGP
.......................GPPGG
...........................G'
............................'UUGA
...............................GA+
.................................+AC
...................................CCAAU
.....................................AAUAGU
.......................................UAGUAGG
.........................................GUAGGG
...........................................AGGGG
..............................................GGGUP
...................................................CG"
....................................................."UUCCU
......................................................UUCCUUC
........................................................CCUUCCU
.....................

## Inspection Results
<b>Inspector 1: Nishal Naicker<b>
- The code looks great! it was readable and the doc strings and comments helped at the more technical parts as well. 
- In terms of running it everything looks great as well!
    
<b> Inspector 2: Jorge Gomez Ortega <b>
- The code is very easy to follow, you did a realy good job at compartamentaliztion of the difffrent functions involved 
    with building the different tRNA objects, along with your really detailed in line comments I found absoultuetly no issue with following along. 
- The output looks great, just as it should. When comparing output with the given file the output looks great. 

# Sample Output

In [ ]:
tRNA|Lys|∃UU|Bostaurus|mitochondrial
CACUAAGA"LCUAUAUAGCACPAACCU∃UU6AGUUAGAGAUUGAGAGCCAU"UACUCUCCUUGGUGACCA
CACU
.ACUA
...UAA
....AAG
.......A"
.........L
..........CUAU
............AUAU
..............AUAG
...............UAGC
.................GCA
.....................P
......................AAC
.......................ACCU
...........................∃
..............................6
...............................AGU
................................GUU
.................................UUA
..................................UAGA
...................................AGAGA
......................................GAU
.......................................AUU
........................................UUGA
.........................................UGAG
..........................................GAGAG
............................................GAGC
..............................................GCC
................................................CAU
..................................................U"
..................................................."U
....................................................UAC
.....................................................ACUC
.......................................................UCU
.........................................................UCC
...........................................................CUU
..............................................................GG
...............................................................GUG
.................................................................GAC
..................................................................ACCA